In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd


import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px
from datetime import datetime, timedelta, date


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'

r = redis.Redis(host="127.0.0.1", port=6379, db=0)

In [15]:
virgo_abuseinject = pd.read_excel(
    '/home/server/gli-data-science/ferina/Virgo_Abuse.xlsx'
    ,dtype=object,sheet_name='Inject', names=['account_card'])


# virgo_abuseuse = pd.read_excel(
#     '/home/server/gli-data-science/ferina/Virgo_Abuse.xlsx'
#     ,dtype=object,sheet_name='Use', names=['account_card'])

## load feature and label 
df_sus_all = pd.read_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full_withname1.csv'
    , dtype='object')
# df_sus_all = df_sus_all[(df_sus_all['created_at_d'] >= '2021-12-01') 
#                         & (df_sus_all['created_at_d'] <= '2022-04-17')]

sus_label = pd.read_csv('./suspect_fraud/sus_all_exclude_trx.csv', dtype='object', usecols=[0])


df_label = pd.concat([sus_label, virgo_abuseinject]).drop_duplicates()
df_label['label'] = 1

## merging feature with suspect label
df_train = pd.merge(df_sus_all, df_label, on='account_card', how='left')
df_train['label'] = df_train['label'].fillna(0)

df_train['score_gibberish'] = df_train['score_gibberish'].astype(float).astype(int)
df_train['score_sus_email'] = df_train['score_sus_email'].astype(int)
df_train['createdFromIp_count'] = df_train['createdFromIp_count'].astype(int)
df_train['phoneSub_count'] = df_train['phoneSub_count'].astype(int)
df_train['full_name_sCount'] = df_train['full_name_sCount'].astype(int)
df_train['full_name_eCount'] = df_train['full_name_eCount'].astype(int)

In [26]:
wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',
'full_name_sCount',
'full_name_eCount'
]

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_train, test_size=0.2, random_state=42)

In [25]:
X_train_feature = X_train[wrap_col]

In [ ]:
from sklearn.cluster import KMeans

#Initiating the Agglomerative Clustering model 
AC = KMeans(n_clusters=9, random_state=0)
# AC = KModes(n_clusters=5, init='Huang', n_init=5, verbose=1)

# fit model and predict clusters without PCA
y_train_res = AC.fit_predict(X_train_feature)

X_train["Clusters"]= y_train_res


In [ ]:
li_train = []
for dr in pd.date_range('2022-01-01', '2022-04-06', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member_sel[alfagift_member_sel['created_at_d'] == sel_created]


    dict_register_ip = {}
    r.set("register_ip", json.dumps(dict_register_ip))
    dict_register_prefix_phone = {}
    r.set("register_prefix_phone", json.dumps(dict_register_prefix_phone))
    dict_register_lower_name = {}
    r.set("register_lower_name", json.dumps(dict_register_lower_name))

    li_score_ip = []
    li_score_prefix_phone = []
    li_score_full_name = []
    li_ponta = []
    li_email = []
    score_ip = 0
    score_prefix_phone = 0
    score_full_name = 0
    
    
    for idx, row in alfagift_member_sel_d.iterrows():
        cur_ponta = row['account_card']
        li_ponta.append(cur_ponta)
        
        cur_email = row['email']
        li_email.append(cur_email)
        
        cur_ip = row['createdFromIp']
        cur_prefix_phone = row['phone'][0:9]
        cur_lower_name = row['full_name'].lower()

        student_obj_as_bytes = r.get("register_ip")
        student_obj_as_str = student_obj_as_bytes.decode("utf-8")
        register_ip = json.loads(student_obj_as_str)

        student_obj_as_bytes = r.get("register_prefix_phone")
        student_obj_as_str = student_obj_as_bytes.decode("utf-8")
        register_prefix_phone = json.loads(student_obj_as_str)

        student_obj_as_bytes = r.get("register_lower_name")
        student_obj_as_str = student_obj_as_bytes.decode("utf-8")
        register_lower_name = json.loads(student_obj_as_str)

        
        if cur_ip in register_ip:
            score_ip += 1
        
        if cur_prefix_phone in register_prefix_phone:
            score_prefix_phone += 1
        
        if cur_lower_name in register_lower_name:
            score_full_name += 1

        li_score_ip.append(score_ip)
        li_score_prefix_phone.append(score_prefix_phone)
        li_score_full_name.append(score_full_name)


        
        if cur_ip in register_ip:
            register_ip[cur_ip] += 1
        else:
            register_ip[cur_ip] = 1

        if cur_prefix_phone in register_prefix_phone:
            register_prefix_phone[cur_prefix_phone] += 1
        else:
            register_prefix_phone[cur_prefix_phone] = 1

        if cur_lower_name in register_lower_name:
            register_lower_name[cur_lower_name] += 1
        else:
            register_lower_name[cur_lower_name] = 1

        r.set("register_ip", json.dumps(register_ip))
        r.set("register_prefix_phone", json.dumps(register_prefix_phone))
        r.set("register_lower_name", json.dumps(register_lower_name))

        #break
    student_obj_as_bytes = r.get("register_ip")
    student_obj_as_str = student_obj_as_bytes.decode("utf-8")
    register_ip = json.loads(student_obj_as_str)

    student_obj_as_bytes = r.get("register_prefix_phone")
    student_obj_as_str = student_obj_as_bytes.decode("utf-8")
    register_prefix_phone = json.loads(student_obj_as_str)

    student_obj_as_bytes = r.get("register_lower_name")
    student_obj_as_str = student_obj_as_bytes.decode("utf-8")
    register_lower_name = json.loads(student_obj_as_str)
        
    df_train = alfagift_member_sel_d.copy()

    df_train['score_ip'] = li_score_ip
    df_train['score_prefix_phone'] = li_score_prefix_phone
    df_train['score_full_name'] = li_score_full_name
    
    df_train['sum_score_ip'] = df_train['createdFromIp'].apply(lambda x: register_ip[x])
    df_train['sum_score_prefix_phone'] = df_train['phone'].apply(lambda x: register_prefix_phone[x[0:9]])
    df_train['sum_score_full_name'] = df_train['full_name'].apply(lambda x: register_lower_name[x.lower()])
    df_train['score_gibberish'] = df_train['email'].apply(score_gibberish)
    df_train['score_sus_email'] = df_train['email'].apply(score_sus_email)
    df_train = df_train.drop(['_id','gender','birth_date','marital_status'], axis=1)
    df_train.to_csv(
        '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/train_{}'\
        .format(sel_created),index=False)
    li_train.append(df_train)